## Project: Πειραματική εφαρμογή του RSA στο SageMath

In [2]:
#  Επιλογή δύο μικρών πρώτων p,q

print("Επιλογή δύο μικρών πρώτων p, q (8–10 bits)")
@interact
def διάλεξε_πρώτους(bits=slider(8, 10, 1, default=8, label="Μήκος σε bits:")):
    max_tries = 10
    p = q = None
    for _ in range(max_tries):
        p = random_prime(2^bits, lbound=2^(bits-1))
        q = random_prime(2^bits, lbound=2^(bits-1))
        if p != q:
            break
    else:
        print("⚠️ Απέτυχε η εύρεση διαφορετικών πρώτων μετά από πολλές προσπάθειες.")
        return

    n = p * q
    phi = (p - 1) * (q - 1)

    print(f"✔ Επιλέχθηκαν:")
    print(f"p = {p}, q = {q}")
    print(f"➡ n = p · q = {n}")
    print(f"➡ φ(n) = (p - 1)(q - 1) = {phi}")

    global RSA_p, RSA_q, RSA_n, RSA_phi
    RSA_p, RSA_q, RSA_n, RSA_phi = p, q, n, phi


Επιλογή δύο μικρών πρώτων p, q (8–10 bits)


Interactive function <function διάλεξε_πρώτους at 0x6ffd3e6dacb0> with 1 widget
  bits: TransformIntSlider(val…

In [3]:
#Επιλογή του e (δημόσιο κλειδί)

print("Επιλογή μικρού e τέτοιου ώστε gcd(e, φ(n)) = 1")
@interact
def επιλέγουμε_e(e=input_box(default=3, label="Δώσε τιμή για e:")):
    phi = RSA_phi
    e = Integer(e)

    if gcd(e, phi) == 1:
        print(f"✔ Το e = {e} είναι έγκυρο (σχετικά πρώτος με φ(n) = {phi})")
    else:
        print(f"✘ Το e = {e} δεν είναι σχετικά πρώτος με φ(n) = {phi}. Δοκίμασε άλλη τιμή.")
    
    # Αποθήκευση
    global RSA_e
    RSA_e = e


Επιλογή μικρού e τέτοιου ώστε gcd(e, φ(n)) = 1


Interactive function <function επιλέγουμε_e at 0x6ffd3e69b680> with 1 widget
  e: EvalText(value='3', descript…

In [4]:
#Υπολογισμός του d (ιδιωτικό κλειδί)

print("Υπολογισμός του d ώστε e·d ≡ 1 mod φ(n)")
@interact
def υπολογισμός_d_μαθητή(
    απάντηση_d=input_box(default=0, label="Η απάντησή σου για d (το αντίστροφο του e modulo φ(n))"),
    εμφάνιση_λύσης=checkbox(False, "Εμφάνισε υπολογισμό και σωστή λύση")
):
    try:
        e = RSA_e
        phi = RSA_phi

        σωστό_d = inverse_mod(e, phi)
        μαθητή_d = Integer(απάντηση_d)

        if μαθητή_d == σωστό_d:
            show(html(f"<b style='color:green'>✔ Σωστά!</b> Το d είναι <b>{σωστό_d}</b>."))
        else:
            show(html(
                f"<b style='color:red'>✘ Λάθος.</b> Η απάντησή σου ήταν <b>{μαθητή_d}</b>, "
                f"αλλά δεν ισχύει ότι {e}×{μαθητή_d} ≡ 1 mod {phi}."
            ))

        if εμφάνιση_λύσης:
            print("Υπολογισμός του d:")
            print(f"➡ Δεδομένα: e = {e}, φ(n) = {phi}")
            print(f"➡ Θέλουμε: d τέτοιο ώστε e·d ≡ 1 mod φ(n)")
            print(f"➡ Υπολογισμός: d = inverse_mod({e}, {phi}) = {σωστό_d}")

        # Αποθήκευση για επόμενα βήματα (αν είναι σωστός)
        if μαθητή_d == σωστό_d:
            global RSA_d
            RSA_d = σωστό_d

    except ValueError:
        print("✘ Το e δεν έχει αντίστροφο modulo φ(n). Διάλεξε άλλο e.")
    except NameError:
        print("⚠️ Πρέπει πρώτα να έχουν υπολογιστεί το e και το φ(n).")


Υπολογισμός του d ώστε e·d ≡ 1 mod φ(n)


Interactive function <function υπολογισμός_d_μαθητή at 0x6ffd3e647320> with 2 widgets
  απάντηση_d: EvalText(v…

In [5]:
print(" Έλεγχος δημόσιου και ιδιωτικού κλειδιού από τον μαθητή")
print("1. Δώσε το n = p·q:"),
print("2. Δώσε το φ(n) = (p−1)(q−1):"),
print("3. Δώσε το e (συνήθως μικρός πρώτος που gcd(e, φ(n))=1):"),
print("4. Δώσε το d (αντίστροφο του e mod φ(n)):"),


@interact
def έλεγχος_κλειδιών_με_υπόδειξη(
    εισαγωγή_n=input_box(label="n = p·q:"),
    εισαγωγή_phi=input_box(label="φ(n) = (p−1)(q−1):"),
    εισαγωγή_e=input_box(label="e"),
    εισαγωγή_d=input_box(label="d:"),
    εμφάνιση_λύσης=checkbox(False, "Εμφάνισε σωστά αποτελέσματα και υπολογισμούς")
):
    try:
        p = RSA_p
        q = RSA_q
        σωστό_n = RSA_n
        σωστό_phi = RSA_phi
        σωστό_e = RSA_e
        σωστό_d = RSA_d

        # Μετατροπή και έλεγχοι
        απάντηση_n = Integer(εισαγωγή_n)
        απάντηση_phi = Integer(εισαγωγή_phi)
        απάντηση_e = Integer(εισαγωγή_e)
        απάντηση_d = Integer(εισαγωγή_d)

        n_ok = απάντηση_n == σωστό_n
        phi_ok = απάντηση_phi == σωστό_phi
        e_ok = απάντηση_e == σωστό_e
        d_ok = απάντηση_d == σωστό_d

        if n_ok and phi_ok and e_ok and d_ok:
            show(html("<b style='color:green'>✔ Όλα τα στάδια είναι σωστά!</b>"))
        else:
            feedback = "<b style='color:red'>✘ Κάποια στάδια είναι λάθος:</b><ul>"
            if not n_ok:
                feedback += f"<li><b>n</b> είναι λάθος. Υπολόγισες {απάντηση_n}</li>"
            if not phi_ok:
                feedback += f"<li><b>φ(n)</b> είναι λάθος. Υπολόγισες {απάντηση_phi}</li>"
            if not e_ok:
                feedback += f"<li><b>e</b> είναι λάθος. Υπολόγισες {απάντηση_e}</li>"
            if not d_ok:
                feedback += f"<li><b>d</b> είναι λάθος. Υπολόγισες {απάντηση_d}</li>"
            feedback += "</ul>"
            show(html(feedback))

        if εμφάνιση_λύσης:
            print("📘 Υπολογισμός κλειδιών RSA:")
            print(f"p = {p}, q = {q}")
            print(f"➡ n = p·q = {p}·{q} = {σωστό_n}")
            print(f"➡ φ(n) = (p-1)(q-1) = ({p - 1})·({q - 1}) = {σωστό_phi}")
            print(f"➡ Επιλέγουμε e ώστε gcd(e, φ(n))=1 → π.χ. e = {σωστό_e}")
            print(f"➡ Υπολογίζουμε d ως αντίστροφο του e mod φ(n): d ≡ e⁻¹ mod φ(n) → d = {σωστό_d}")

    except NameError:
        print("⚠️ Πρέπει πρώτα να έχουν υπολογιστεί οι τιμές p και q (Βήμα 1).")


 Έλεγχος δημόσιου και ιδιωτικού κλειδιού από τον μαθητή
1. Δώσε το n = p·q:
2. Δώσε το φ(n) = (p−1)(q−1):
3. Δώσε το e (συνήθως μικρός πρώτος που gcd(e, φ(n))=1):
4. Δώσε το d (αντίστροφο του e mod φ(n)):


Interactive function <function έλεγχος_κλειδιών_με_υπόδειξη at 0x6ffd3e4beb90> with 5 widgets
  εισαγωγή_n: Ev…

In [6]:
print("Κρυπτογράφηση μηνύματος M")
print("Μήνυμα M, ακέραιος < n):"),
print("Η απάντησή σου για C (C = M^e mod n):"),
@interact
def αξιολόγηση_κρυπτογράφησης(
    M=input_box(default=12, label="M:"),
    απάντηση=input_box(default=0, label="C:"),
    εμφάνιση_λύσης=checkbox(False, "Εμφάνισε σωστή λύση και υπολογισμό")
):
    try:
        n = RSA_n
        e = RSA_e
        M = Integer(M)

        if M >= n:
            show(html("<b style='color:red'>❌ Το M πρέπει να είναι μικρότερο από n!</b>"))
            return

        σωστό_C = power_mod(M, e, n)
        μαθητή_C = Integer(απάντηση)

        if μαθητή_C == σωστό_C:
            show(html(f"<b style='color:green'>✔ Σωστά!</b> Το C είναι <b>{σωστό_C}</b>."))
        else:
            show(html(f"<b style='color:red'>✘ Λάθος.</b> Η απάντησή σου ήταν <b>{μαθητή_C}</b>, αλλά το σωστό C είναι διαφορετικό."))

        if εμφάνιση_λύσης:
            print("📘 Υπολογισμός Κρυπτογραφήματος:")
            print(f"➡ M = {M}")
            print(f"➡ Δημόσιο κλειδί: (n, e) = ({n}, {e})")
            print(f"➡ Υπολογισμός: C = M^e mod n = {M}^{e} mod {n}")
            print(f"➡ Τελικό αποτέλεσμα: C = {σωστό_C}")

        # Αποθήκευση για επόμενα βήματα
        global RSA_M, RSA_C
        RSA_M, RSA_C = M, σωστό_C

    except NameError:
        print("⚠️ Οι μεταβλητές RSA_n ή RSA_e δεν έχουν οριστεί (πρέπει να προηγηθεί ο υπολογισμός κλειδιών).")


Κρυπτογράφηση μηνύματος M
Μήνυμα M, ακέραιος < n):
Η απάντησή σου για C (C = M^e mod n):


Interactive function <function αξιολόγηση_κρυπτογράφησης at 0x6ffd3e475c20> with 3 widgets
  M: EvalText(value…

In [7]:
print("Αποκρυπτογράφηση μηνύματος")
print("Δώσε το C (κρυπτογράφημα από προηγούμενη κρυπτογράφηση):")

@interact
def αποκρυπτογράφηση_με_έλεγχο(
    εισαγόμενο_C=input_box(default=0, label="C:"),
    εμφάνιση_λύσης=checkbox(False, "Εμφάνισε σωστή λύση")
):
    try:
        C = Integer(εισαγόμενο_C)
        M_αποκρυπτογραφημένο = power_mod(C, RSA_d, RSA_n)

        if C == RSA_C:
            if M_αποκρυπτογραφημένο == RSA_M:
                show(html(f"<b style='color:green'>✔ Σωστά!</b> Το C αποκρυπτογραφήθηκε σωστά σε M = <b>{M_αποκρυπτογραφημένο}</b>."))
            else:
                show(html(f"<b style='color:red'>❌ Λάθος αποκρυπτογράφηση.</b> Το αποτέλεσμα δεν ταιριάζει με το αρχικό M."))
        else:
            show(html(f"<b style='color:orange'>⚠️ Προσοχή!</b> Το C που έδωσες (<b>{C}</b>) <b>δεν είναι</b> αυτό που υπολογίστηκε στην κρυπτογράφηση (<b>{RSA_C}</b>), οπότε η αποκρυπτογράφηση δεν έχει νόημα ως έλεγχος."))

        if εμφάνιση_λύσης:
            print("Υπολογισμός:")
            print(f"➡ C = {C}")
            print(f"➡ d = {RSA_d}, n = {RSA_n}")
            print(f"➡ M = C^d mod n = {C}^{RSA_d} mod {RSA_n} = {M_αποκρυπτογραφημένο}")

    except NameError:
        print("⚠️ Πρέπει πρώτα να έχουν υπολογιστεί τα κλειδιά και το C από την κρυπτογράφηση.")


Αποκρυπτογράφηση μηνύματος
Δώσε το C (κρυπτογράφημα από προηγούμενη κρυπτογράφηση):


Interactive function <function αποκρυπτογράφηση_με_έλεγχο at 0x6ffd3e4753b0> with 2 widgets
  εισαγόμενο_C: Ev…

In [8]:
print("Πειραματισμός: Τι συμβαίνει αν το p ή το q δεν είναι πρώτος αριθμός;")
print("Δοκίμασε σύνθετο p και σύνθετο q:")

@interact
def σύνθετος_παράγοντας(p=input_box(default=15, label="p:"),
                        q=input_box(default=21, label="q:")):
    try:
        n = p * q
        phi = (p - 1)*(q - 1)
        e = 3
        d = inverse_mod(e, phi)
        print("❗ Παρ' ότι p, q δεν είναι πρώτοι, το RSA μοιάζει να δουλεύει...")
        print("✘ Όμως δεν είναι ασφαλές: ο επιτιθέμενος μπορεί εύκολα να παραγοντοποιήσει n.")
    except:
        print("✔ Δεν ορίζεται αντίστροφος. Το RSA αποτυγχάνει όπως πρέπει σε σύνθετους αριθμούς.")


Πειραματισμός: Τι συμβαίνει αν το p ή το q δεν είναι πρώτος αριθμός;
Δοκίμασε σύνθετο p και σύνθετο q:


Interactive function <function σύνθετος_παράγοντας at 0x6ffd3e402950> with 2 widgets
  p: EvalText(value='15',…